In [ ]:
# ! pip install skforecast

In [ ]:
# Tratamiento de datos
# ==============================================================================
import numpy as np
import pandas as pd
import xgboost as xgb

from scipy import stats

# Gráficos
# ==============================================================================
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
plt.rcParams['lines.linewidth'] = 1.5
plt.rcParams['font.size'] = 10

# Modelado y Forecasting
# ==============================================================================
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Lasso, Ridge
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier

from skforecast.ForecasterAutoreg import ForecasterAutoreg
from skforecast.ForecasterAutoregCustom import ForecasterAutoregCustom
from skforecast.ForecasterAutoregDirect import ForecasterAutoregDirect
from skforecast.model_selection import grid_search_forecaster
from skforecast.model_selection import backtesting_forecaster
from skforecast.utils import save_forecaster

from sklearn.neural_network import MLPRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import  KNeighborsRegressor
from sklearn import linear_model
from sklearn.model_selection import KFold, cross_val_score, train_test_split

from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.ar_model import AutoReg

from skforecast.utils import load_forecaster

import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping
from keras import optimizers,callbacks
from keras.models import Sequential, Model
from keras.layers import Conv1D, Conv2D,MaxPooling1D
from keras.layers import Dense, LSTM, RepeatVector, TimeDistributed, Flatten, Dropout, MultiHeadAttention, GlobalAveragePooling1D, Input
from keras import optimizers

# Calculo de metricas
# ==============================================================================
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error
from sklearn import metrics

# Configuración warnings
# ==============================================================================
import warnings
pd.options.mode.chained_assignment = None  # default='warn'
warnings.filterwarnings("ignore")
# warnings.filterwarnings('ignore')

In [ ]:
def evaluacion_metrica(y_true, y_pred):

    def mean_absolute_percentage_error(y_true, y_pred):
        y_true, y_pred = np.array(y_true), np.array(y_pred)
        return np.mean(np.abs((y_true - y_pred) / y_true)) * 100
    print('Evaluation metric results:-')
    print(f'R2: {metrics.r2_score(y_true, y_pred)}',end='\n\n')
    print(f'MSE: {metrics.mean_squared_error(y_true, y_pred)}')
    print(f'MAE: {metrics.mean_absolute_error(y_true, y_pred)}')



In [ ]:
# Carga de datos
# ==============================================================================
folder = ""
parametro = 'temperatura'
output=parametro
inputs = [
          'fecha', 'temperatura',
          'ambiente'
          ]
usecols = inputs.copy()

datos = pd.read_csv(folder + "lagoon_hourly_filled" + ".csv",sep=',', usecols=usecols)

# Preparación del dato
# ==============================================================================
datos['fecha'] = pd.to_datetime(datos['fecha'], format='%d/%m/%Y %H:%M')
datos = datos.set_index('fecha')
datos = datos.rename(columns={'x': 'y'})
datos = datos.asfreq('H')
datos = datos.sort_index()

datos.info()

datos.head()

In [ ]:
#datos["ta-1"] = datos["ambiente"].shift(1)
#datos["ta-2"] = datos["ambiente"].shift(2)
#datos["ta-3"] = datos["ambiente"].shift(3)
#datos["ta-4"] = datos["ambiente"].shift(4)
#datos["ta-5"] = datos["ambiente"].shift(5)
datos["ta-24"] = datos["ambiente"].shift(24)
datos["ta-36"] = datos["ambiente"].shift(36)
datos["ta-48"] = datos["ambiente"].shift(48)
datos["ta-96"] = datos["ambiente"].shift(96)
datos["ta-120"] = datos["ambiente"].shift(120)

#datos = datos.drop(columns=["ambiente"])
datos = datos.dropna()

datos.tail(10)

## MODELADO

In [ ]:
def rmsle_cv(model,x,y):
    kf = KFold(5, shuffle=True, random_state=42).get_n_splits(x.values)
    rmse= cross_val_score(model, x.values, y, scoring=None, cv = kf)
    return(rmse)

## (Linear Regression)

In [ ]:
# y_train = datos["temperatura"].loc[:].values
# datos_train = datos.drop(columns=['temperatura'])

# lm = LinearRegression()
# model = lm.fit(datos_train,y_train)

# res_lineal = rmsle_cv(lm,datos_train,y_train)
# print(res_lineal)

# l = plt.plot(lm.predict(datos_train), 'o', color='blue', label="Estimado", marker='x')
# plt.setp(l, markersize=3)

# plt.title('Estimado')
# q = plt.plot(y_train, 'o', color='red', label="Original")
# plt.setp(q, markersize=3)
# category_names = ['Estimado', 'Original']
# legend = plt.legend(ncol=len(category_names), bbox_to_anchor=(0, 1),
#               loc='lower left', fontsize='medium')
# plt.show()

## LR, Lasso, KNN, MLP, DT, RF

In [ ]:
def ML_algorithms(algo):
    if algo == 'KNN':
      model = KNeighborsRegressor(3, weights='distance') # RDregresor
      return model

    if algo=="MLP":
      #(64, 32)	0.01	200.0
      model = MLPRegressor(hidden_layer_sizes=(32,32), learning_rate_init=0.01, max_iter=400, random_state=1)
      return model

    if algo=='DT':
      model = DecisionTreeRegressor(max_depth=16, min_samples_split=2,random_state=0)
      return model

    if algo=="RFR":
      #antes 16 y 4
      model =  RandomForestRegressor(random_state=0, n_estimators=8, min_samples_split=16)
      return model

    if algo=="LR":
        #antes 16 y 4
        model =  LinearRegression()
        return model

    if algo=="Lasso":
      #antes 16 y 4
      model =  Lasso()
      return model

    if algo=="SVM":
      #antes 16 y 4
      model =  SVR()
      return model

In [ ]:
algoritmos = ['LR','Lasso','KNN', 'MLP', 'DT', 'RFR', 'SVM']
cv_scores = pd.DataFrame()
dfe=datos.copy()

inputs_temp = [
          'ambiente','ta-24','ta-36','ta-48','ta-96','ta-120'
          ]

for algo in algoritmos:

    print("Algoritmo", algo)
    dfwork = dfe.copy()

    x = dfwork[inputs_temp].values
    y = dfwork[output].values

    lm = ML_algorithms(algo)

    kf = KFold(5, shuffle=True, random_state=42)
    icv=0
    acc_score = []
    for train, test in kf.split(x):
      X_train, X_test, y_train, y_test = x[train], x[test], y[train], y[test]
      model = lm.fit(X_train,y_train)
      ypred = model.predict(X_test)

      r2 = r2_score(y_test, ypred)
      mae = mean_absolute_error(y_test, ypred)
      mse = mean_squared_error(y_test, ypred)
      icv = icv + 1

      result = {"Algorithm": algo, "cv": icv, "r2": r2, "MSE": mse, 'MAE': mae}

      # Assuming cv_scores is an existing DataFrame
      new_entry = pd.DataFrame([result])
      cv_scores = pd.concat([cv_scores, new_entry], ignore_index=True)
      print(result)

print(cv_scores)

r = cv_scores.groupby(['Algorithm']).mean()
print(r)

## KNN, RF, SVM, DT, LR

In [ ]:
entreno = 120

datos_entreno = datos.iloc[:-entreno] # PARA TENER UNA COPIA DEL ORIGINAL Y MODIFICAR LA COPIA

X_train = datos[inputs_temp].iloc[:-entreno]
y_train = datos[output].iloc[:-entreno]
X_test = datos[inputs_temp].iloc[-entreno:]
y_test = datos[output].iloc[-entreno:]

In [ ]:
# model_LR =  linear_model.LinearRegression().fit(X_train,y_train)
# model_KN = KNeighborsRegressor(2,weights='distance').fit(X_train,y_train)
# #model_LR = Lasso(alpha=1.0).fit(X_train,y_train) #LASSO y RIDGE muy malos
# #model_LR = Ridge(alpha=1.0).fit(X_train,y_train) #LASSO y RIDGE muy malos
# #model_LR = SVR(kernel='linear', C=1.0, epsilon=0.1).fit(X_train,y_train)
# model_RF = RandomForestRegressor(random_state=0, n_estimators=8, min_samples_split=16).fit(X_train,y_train)

# ypred = model_LR.predict(X_test)

# predicciones = pd.DataFrame() #ESTO PARA CREAR UN DATAFRAME SOLO CON LAS PREDICCIONES, PARA PINTAR LUEGO
# predicciones_fecha = pd.DataFrame() #ESTO PARA CREAR UN DATAFRAME SOLO CON LAS PREDICCIONES, PARA PINTAR LUEGO

# predicciones["pred"] = ypred

# r2 = r2_score(y_test, ypred)
# mae = mean_absolute_error(y_test, ypred)
# mse = mean_squared_error(y_test, ypred)

# predicciones_fecha = X_test.copy()
# predicciones_fecha["temperatura"] = predicciones["pred"].values

# print(predicciones_fecha)

In [ ]:
def KNN(nestimators):
      model = KNeighborsRegressor(nestimators,weights='distance') # RDregresor
      return model

def DT(depth, mss):
  model = DecisionTreeRegressor(max_depth=depth, min_samples_split=mss,random_state=0)
  return model

def RFR(nestimators, mss):
  model =  RandomForestRegressor(random_state=0, n_estimators=nestimators, min_samples_split=mss)
  return model

def SVR_V(kernel, epsilon):
  model = SVR(kernel = kernel, C = 1.0, epsilon= epsilon)
  return model

In [ ]:
dfe=datos.copy()

max_depth = [4,8,10,16,30,60,120] #DT
mss = [2,4,8,16,32,64] #DT , RFR
nstimator=[1,2,3,4,5,6,7,8,9,10] # RFR, KNN
kernels = ['linear','poly','rbf','simoid','precomputed'] # SVR
epsilons = [0.1,0.2,0.5,1.0] # SVR


best_model = None
best_score = 0
cv_scores_DT = pd.DataFrame()
cv_scores_KNN = pd.DataFrame()
cv_scores_RF = pd.DataFrame()
cv_scores_SVR = pd.DataFrame()
cv_scores_best = pd.DataFrame()

#================================ RF ==================================

for ns in nstimator:
    for ms in mss:

        dfwork = dfe.copy()

        x = dfwork[inputs_temp].values
        y = dfwork[output].values

        model = RFR(ns, ms)
        cv = KFold(n_splits=5, random_state=1, shuffle=True)
        scores = cross_val_score(model, x, y, cv=cv, n_jobs=-1)

        if (scores.mean() > best_score):
            best_score = scores.mean()
            best_model = model
            result = {"Parametro1": ns, "Parametro2": ms, 'score': scores.mean()}

            # Assuming cv_scores_DT is an existing DataFrame
            new_entry = pd.DataFrame([result])  # Create a new DataFrame with a single row
            cv_scores_RF = pd.concat([cv_scores_RF, new_entry], ignore_index=True)
print("=============== RF =================")
print(best_model)
print(best_score)
print(cv_scores_RF.sort_values('score', ascending=False))

#================================ DT ==================================

best_model = None
best_score = 0

for ns in max_depth:

    for ms in mss:

        dfwork = dfe.copy()

        x = dfwork[inputs_temp].values
        y = dfwork[output].values

        model = DT(ns, ms)
        cv = KFold(n_splits=5, random_state=1, shuffle=True)
        scores = cross_val_score(model, x, y, cv=cv, n_jobs=-1)

        if (scores.mean() > best_score):
            best_score = scores.mean()
            best_model = model
            result = {"Parametro1": ns, "Parametro2": ms, 'score': scores.mean()}

            # Assuming cv_scores_DT is an existing DataFrame
            new_entry = pd.DataFrame([result])  # Create a new DataFrame with a single row
            cv_scores_DT = pd.concat([cv_scores_DT, new_entry], ignore_index=True)
print("=============== DT =================")
print(best_model)
print(best_score)
print(cv_scores_DT.sort_values('score', ascending=False))


#================================ KNN ==================================

best_model = None
best_score = 0

for nstim in nstimator:
    for ms in mss:

      dfwork = dfe.copy()

      x = dfwork[inputs_temp].values
      y = dfwork[output].values

      model = KNN(nstim)
      cv = KFold(n_splits=5, random_state=1, shuffle=True)
      scores = cross_val_score(model, x, y, cv=cv, n_jobs=-1)

      if (scores.mean() > best_score):
          best_score = scores.mean()
          best_model = model
          result = {"Parametro1": ns, "Parametro2": ms, 'score': scores.mean()}
          new_entry = pd.DataFrame([result])  # Create a new DataFrame with a single row
          cv_scores_KNN = pd.concat([cv_scores_KNN, new_entry], ignore_index=True)

print("=============== KNN =================")
print(best_model)
print(best_score)
print(cv_scores_KNN.sort_values('score', ascending=False))

#================================ SVR ==================================

# best_model = None
# best_score = 0

# for kernel in kernels:
#     for epsilon in epsilons:

#       dfwork = dfe.copy()

#       x = dfwork[inputs_temp].values
#       y = dfwork[output].values

#       model = SVR_V(kernel, epsilon)
#       cv = KFold(n_splits=5, random_state=1, shuffle=True)
#       scores = cross_val_score(model, x, y, cv=cv, n_jobs=-1)

#       if (scores.mean() > best_score):
#           best_score = scores.mean()
#           best_model = model
#           result = {"Parametro1": ns, "Parametro2": ms, 'score': scores.mean()}
#           new_entry = pd.DataFrame([result])  # Create a new DataFrame with a single row
#           cv_scores_SVR = pd.concat([cv_scores_SVR, new_entry], ignore_index=True)

# print("=============== SVR =================")
# print(best_model)
# print(best_score)
# print(cv_scores_SVR.sort_values('score', ascending=False))

### Modelos definitivos

In [ ]:
inputs_temp = [
          'ambiente','ta-24','ta-36','ta-48','ta-96','ta-120'
          ]
outputs = ['temperatura']


X = datos[inputs_temp]
y = datos[output]


model_LR = LinearRegression().fit(X, y)
model_KNN = KNeighborsRegressor(n_neighbors=3, weights='distance').fit(X, y)
model_DT = DecisionTreeRegressor(max_depth=16, min_samples_split=32, random_state=0).fit(X, y)
model_RF =  RandomForestRegressor(n_estimators=10, random_state=0).fit(X, y)
# model_SVR = SVR(kernel = kernel, C = 1.0, epsilon= epsilon).fit(X, y)

# REDES NEURONALES

## CNN

In [ ]:
def create_model(opt, loss="mse"):
  model = Sequential()
  # opt = tf.optimizers.Adam(learning_rate=0.1)
  # model.add(Dense(32, activation="relu"))
  model.add(Conv1D(128, 2, activation="sigmoid", input_shape=(6, 1)))
  model.add(Dense(64, activation="sigmoid"))
  model.add(Dense(32, activation="relu"))
  model.add(Flatten())
  model.add(Dense(1))
  model.compile(loss=loss, optimizer='adam')
  # model.summary()
  return model

#### Aquí con todos los datos

In [ ]:
results = pd.DataFrame()

dfwork = dfe.copy()
x = dfwork[inputs_temp].to_numpy()
y = dfwork[output].to_numpy()
x = x.reshape(x.shape[0], x.shape[1], 1)
opt = "adam"

for i in range(0,5):
    xtrain, xtest, ytrain, ytest= train_test_split(x, y, train_size=0.8)
    print(len(xtrain), len(ytrain))
    model = create_model(opt)
    # if show_model:
    #  model.summary()
    #  show_model=False
    model.fit(xtrain, ytrain, batch_size=16, epochs=500, verbose=0)
    #print(xtest)
    ypred = model.predict(xtest)

    r2 = r2_score(ytest, ypred)
    mae = mean_absolute_error(ytest, ypred)
    mse = mean_squared_error(ytest, ypred)
    # plot_scatter(ytest, ypred)
    mr = { 'cv': i+1,
          'R2': r2,
          'MSE':mse,
          'MAE':mae}

    new_entry = pd.DataFrame([mr])  # Create a new DataFrame with a single row
    results = pd.concat([results, new_entry], ignore_index=True)

print(results)

r = results.mean()
print(r)

#### Aquí con los datos de train y test

In [ ]:
results = pd.DataFrame()

dfwork = dfe.copy()
x = dfwork[inputs_temp].to_numpy()
y = dfwork[output].to_numpy()
x = x.reshape(x.shape[0], x.shape[1], 1)
opt = "adam"


model_CNN = create_model(opt)
model_CNN.fit(x, y, batch_size=24, epochs=500, verbose=1)
#print(xtest)
# ypred = model_CNN.predict(X_test)

## RNN

In [ ]:
def create_model_RNN(opt, loss="mse"):
  model_RNN = tf.keras.Sequential([
    tf.keras.layers.SimpleRNN(32, return_sequences=True, input_shape=(6, 1)),  # Capa RNN 1
    tf.keras.layers.SimpleRNN(64, return_sequences=True),  # Capa RNN 2
    tf.keras.layers.SimpleRNN(128, return_sequences=True),  # Capa RNN 3
    tf.keras.layers.SimpleRNN(64, return_sequences=True),  # Capa RNN 4
    tf.keras.layers.SimpleRNN(32),  # Capa RNN 5
    tf.keras.layers.Dense(1)  # Capa de salida con una neurona
])

  model_RNN.compile(loss=loss, optimizer=opt)
  return model_RNN

In [ ]:
results = pd.DataFrame()

dfwork = dfe.copy()
x = dfwork[inputs_temp].to_numpy()
y = dfwork[output].to_numpy()
x = x.reshape(x.shape[0], x.shape[1], 1)
opt = "adam"

for i in range(0,5):

    xtrain, xtest, ytrain, ytest= train_test_split(x, y, train_size=0.8)
    print(len(xtrain), len(ytrain))
    model = create_model_RNN(opt)
    # if show_model:
    #  model.summary()
    #  show_model=False
    model.fit(xtrain, ytrain, epochs=100, verbose=0)
    #print(xtest)
    ypred = model.predict(xtest)

    r2 = r2_score(ytest, ypred)
    mae = mean_absolute_error(ytest, ypred)
    mse = mean_squared_error(ytest, ypred)
    # plot_scatter(ytest, ypred)
    mr = { 'cv': i,
          'MSE':mse,
          'R2': r2,
          'MAE':mae}

    new_entry = pd.DataFrame([mr])  # Create a new DataFrame with a single row
    results = pd.concat([results, new_entry], ignore_index=True)

print(results)

r = results.mean()
print(r)

#### Aquí con los datos de train y test

In [ ]:
dfwork = dfe.copy()
x = dfwork[inputs_temp].to_numpy()
y = dfwork[output].to_numpy()
x = x.reshape(x.shape[0], x.shape[1], 1)

# Crear una RNN de 5 capas
model_RNN = create_model_RNN('adam')

# Entrenar la RNN
model_RNN.fit(x, y, epochs=100, verbose=1)

# Realizar predicciones
# predicted = model_RNN.predict(X_test)

# # Imprimir las predicciones
# print("Predicciones:")
# print(predicted)

In [ ]:
# Carga de datos
# ==============================================================================
folder = ""
parametro = 'temp'
output=parametro
inputs = [
          'fecha', 'ambiente'
          ]

usecols = inputs.copy()

datos_hist = pd.read_csv(folder + "historical_air_temp" + ".csv",sep=',', usecols=usecols)


# Preparación del dato
# ==============================================================================
datos_hist['fecha'] = pd.to_datetime(datos_hist['fecha'], format='%d/%m/%Y')
datos_hist = datos_hist.set_index('fecha')
datos_hist = datos_hist.rename(columns={'x': 'y'})
datos_hist = datos_hist.asfreq('D')
datos_hist = datos_hist.sort_index()

datos_hist.info()

#datos = datos.iloc[:-180]

datos_hist.head()

In [ ]:
# dfwork = datos_hist[datos_hist['NOMBRE']=="LAGUNA DE LA MATA"] # CAMBIAR

dfwork = datos_hist.dropna()

dfwork.info()

fig, ax = plt.subplots(figsize=(20, 8))
dfwork['ambiente'].iloc[:35].plot(ax=ax, label='Temp_Laguna_mata')

In [ ]:
dfwork["ta-24"] = dfwork["ambiente"].shift(1)
dfwork["ta-36"] = dfwork["ambiente"].shift(2)
dfwork["ta-48"] = dfwork["ambiente"].shift(3)
dfwork["ta-96"] = dfwork["ambiente"].shift(4)
dfwork["ta-120"] = dfwork["ambiente"].shift(5)

dfwork = dfwork.dropna()

#X_test.info()
dfwork.info()

In [ ]:
ypred_CNN = model_CNN.predict(dfwork)
ypred_RNN = model_RNN.predict(dfwork)
ypred_LR = model_LR.predict(dfwork)
ypred_DT = model_DT.predict(dfwork)
ypred_KN = model_KNN.predict(dfwork)
ypred_RF = model_RF.predict(dfwork)

In [ ]:
dfwork["temp_agua_CNN"] = ypred_CNN
dfwork["temp_agua_RNN"] = ypred_RNN
dfwork["temp_agua_LR"] = ypred_LR
dfwork["temp_agua_DT"] = ypred_DT
dfwork["temp_agua_KN"] = ypred_KN
dfwork["temp_agua_RF"] = ypred_RF

In [ ]:
plt.figure(figsize=(20,12))

plt.title('Y_predict')
l1 = plt.plot(dfwork["temp_agua_CNN"].iloc[:365], color='blue', label="Temperatura agua CNN")
plt.setp(l1, markersize=8)

l2 = plt.plot(dfwork["temp_agua_RNN"].iloc[150:255], color='green', label="Temperatura agua RNN")
plt.setp(l2, markersize=8)

l3 = plt.plot(dfwork["temp_agua_LR"].iloc[:365], color='red', label="Temperatura agua LR")
plt.setp(l3, markersize=8)

l3 = plt.plot(dfwork["temp_agua_DT"].iloc[:365], color='black', label="Temperatura agua DT")
plt.setp(l3, markersize=8)

l4 = plt.plot(dfwork["temp_agua_KN"].iloc[:365], color='green', label="Temperatura agua KN")
plt.setp(l4, markersize=8)

l5 = plt.plot(dfwork["temp_agua_RF"].iloc[150:255], color='grey', label="Temperatura agua RF")
plt.setp(l5, markersize=8)

plt.title('Predicción con CNN')
q = plt.plot(dfwork["ambiente"].iloc[:365] , color='black', label="Temperatura ambiente")
plt.setp(q, markersize=8)
category_names = ['Estimado', 'Original']
legend = plt.legend(ncol=len(category_names), bbox_to_anchor=(0, 1),
              loc='lower left', fontsize='medium')
plt.show()

# evaluacion_metrica(y_true = y_test.values,y_pred = ypred)

In [ ]:
datos_reshape = datos.resample('1d').mean()

In [ ]:
fig, ax = plt.subplots(figsize=(20, 8))
datos_reshape['temperatura'].plot(ax=ax, label='train')
datos_reshape['ambiente'].plot(ax=ax, label='train')

In [ ]:
plt.style.use('default')

In [ ]:
colores = ["cornflowerblue", "seagreen","indigo","red","chocolate","orchid","darkgray","gold"]

plt.figure(figsize=(15,7))


l1 = plt.plot(dfwork["temp_agua_CNN"].iloc[:365], color='cornflowerblue', label="Tª agua CNN")
plt.setp(l1, markersize=8)

q = plt.plot(dfwork["ambiente"].iloc[:365] , color='darkgray', label="Tª ambiente")

# Configuración de los ejes y títulos
plt.xlabel('Fecha', fontsize=14)
plt.ylabel('Temperatura (ºC)', fontsize=14)
plt.title(f'Reconstrucción histórica 1947 - CNN', fontsize=16)

# Quitar los bordes superior y derecho
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Ajustar el tamaño de la leyenda
plt.legend(fontsize=12)

# Ajustar el tamaño de las etiquetas de los ejes
ax.tick_params(axis='both', which='major', labelsize=12)

# Configurar la cuadrícula
ax.grid(True, which='both', linestyle='--', linewidth=0.7, color='grey', alpha = 0.5)

# Guardar el gráfico en una carpeta con buena resolución
plt.savefig(f"recons_1947_CNN.png", dpi=300, bbox_inches='tight')
plt.show()

#############################################################################

plt.figure(figsize=(15,7))

l3 = plt.plot(dfwork["temp_agua_LR"].iloc[:365], color='seagreen', label="Tª agua LR")
plt.setp(l3, markersize=8)


q = plt.plot(dfwork["ambiente"].iloc[:365] , color='darkgray', label="Tª ambiente")

# Configuración de los ejes y títulos
plt.xlabel('Fecha', fontsize=14)
plt.ylabel('Temperatura (ºC)', fontsize=14)
plt.title(f'Reconstrucción histórica 1947 - LR', fontsize=16)

# Quitar los bordes superior y derecho
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Ajustar el tamaño de la leyenda
plt.legend(fontsize=12)

# Ajustar el tamaño de las etiquetas de los ejes
ax.tick_params(axis='both', which='major', labelsize=12)

# Configurar la cuadrícula
ax.grid(True, which='both', linestyle='--', linewidth=0.7, color='grey', alpha = 0.5)

# Guardar el gráfico en una carpeta con buena resolución
plt.savefig(f"recons_1947_LR.png", dpi=300, bbox_inches='tight')
plt.show()

#############################################################################

plt.figure(figsize=(15,7))

plt.title('Y_predict')
l1 = plt.plot(dfwork["temp_agua_CNN"].iloc[:365], color='cornflowerblue', label="Tª agua CNN")
plt.setp(l1, markersize=8)

l3 = plt.plot(dfwork["temp_agua_LR"].iloc[:365], color='seagreen', label="Tª agua LR")
plt.setp(l3, markersize=8)

q = plt.plot(dfwork["ambiente"].iloc[:365] , color='darkgray', label="Tª ambiente")

# Configuración de los ejes y títulos
plt.xlabel('Fecha', fontsize=14)
plt.ylabel('Temperatura (ºC)', fontsize=14)
plt.title(f'Reconstrucción histórica 1947 - CNN y LR', fontsize=16)

# Quitar los bordes superior y derecho
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Ajustar el tamaño de la leyenda
plt.legend(fontsize=12)

# Ajustar el tamaño de las etiquetas de los ejes
ax.tick_params(axis='both', which='major', labelsize=12)

# Configurar la cuadrícula
ax.grid(True, which='both', linestyle='--', linewidth=0.7, color='grey', alpha = 0.5)

# Guardar el gráfico en una carpeta con buena resolución
plt.savefig(f"recons_1947_CNN_LR.png", dpi=300, bbox_inches='tight')
plt.show()

#############################################################################

plt.figure(figsize=(15,7))

l3 = plt.plot(dfwork["temp_agua_DT"].iloc[:365], color='red', label="Tª agua DT")
plt.setp(l3, markersize=8)

q = plt.plot(dfwork["ambiente"].iloc[:365] , color='darkgray', label="Tª ambiente")

# Configuración de los ejes y títulos
plt.xlabel('Fecha', fontsize=14)
plt.ylabel('Temperatura (ºC)', fontsize=14)
plt.title(f'Reconstrucción histórica 1947 - DT', fontsize=16)

# Quitar los bordes superior y derecho
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Ajustar el tamaño de la leyenda
plt.legend(fontsize=12)

# Ajustar el tamaño de las etiquetas de los ejes
ax.tick_params(axis='both', which='major', labelsize=12)

# Configurar la cuadrícula
ax.grid(True, which='both', linestyle='--', linewidth=0.7, color='grey', alpha = 0.5)

# Guardar el gráfico en una carpeta con buena resolución
plt.savefig(f"recons_1947_DT.png", dpi=300, bbox_inches='tight')
plt.show()

#############################################################################

plt.figure(figsize=(15,7))


l4 = plt.plot(dfwork["temp_agua_KN"].iloc[:365], color='gold', label="Tª agua KNN")
plt.setp(l4, markersize=8)


q = plt.plot(dfwork["ambiente"].iloc[:365] , color='darkgray', label="Tª ambiente")


# Configuración de los ejes y títulos
plt.xlabel('Fecha', fontsize=14)
plt.ylabel('Temperatura (ºC)', fontsize=14)
plt.title(f'Reconstrucción histórica 1947 - KNN', fontsize=16)

# Quitar los bordes superior y derecho
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Ajustar el tamaño de la leyenda
plt.legend(fontsize=12)

# Ajustar el tamaño de las etiquetas de los ejes
ax.tick_params(axis='both', which='major', labelsize=12)

# Configurar la cuadrícula
ax.grid(True, which='both', linestyle='--', linewidth=0.7, color='grey', alpha = 0.5)

# Guardar el gráfico en una carpeta con buena resolución
plt.savefig(f"recons_1947_KNN.png", dpi=300, bbox_inches='tight')
plt.show()

#############################################################################

plt.figure(figsize=(15,7))

l5 = plt.plot(dfwork["temp_agua_RF"].iloc[:365], color='indigo', label="Tª agua RF")
plt.setp(l5, markersize=8)

q = plt.plot(dfwork["ambiente"].iloc[:365] , color='darkgray', label="Tª ambiente")

# Configuración de los ejes y títulos
plt.xlabel('Fecha', fontsize=14)
plt.ylabel('Temperatura (ºC)', fontsize=14)
plt.title(f'Reconstrucción histórica 1947 - RF', fontsize=16)

# Quitar los bordes superior y derecho
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Ajustar el tamaño de la leyenda
plt.legend(fontsize=12)

# Ajustar el tamaño de las etiquetas de los ejes
ax.tick_params(axis='both', which='major', labelsize=12)

# Configurar la cuadrícula
ax.grid(True, which='both', linestyle='--', linewidth=0.7, color='grey', alpha = 0.5)

# Guardar el gráfico en una carpeta con buena resolución
plt.savefig(f"recons_1947_RF.png", dpi=300, bbox_inches='tight')
plt.show()

#############################################################################

plt.figure(figsize=(15,7))

l3 = plt.plot(dfwork["temp_agua_DT"].iloc[:365], color='red', label="Tª agua DT")
plt.setp(l3, markersize=8)

l4 = plt.plot(dfwork["temp_agua_KN"].iloc[:365], color='gold', label="Tª agua KNN")
plt.setp(l4, markersize=8)

l5 = plt.plot(dfwork["temp_agua_RF"].iloc[:365], color='indigo', label="Tª agua RF")
plt.setp(l5, markersize=8)

q = plt.plot(dfwork["ambiente"].iloc[:365] , color='darkgray', label="Tª ambiente")

# Configuración de los ejes y títulos
plt.xlabel('Fecha', fontsize=14)
plt.ylabel('Temperatura (ºC)', fontsize=14)
plt.title(f'Reconstrucción histórica 1947 - DT, KNN y RF', fontsize=16)

# Quitar los bordes superior y derecho
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Ajustar el tamaño de la leyenda
plt.legend(fontsize=12)

# Ajustar el tamaño de las etiquetas de los ejes
ax.tick_params(axis='both', which='major', labelsize=12)

# Configurar la cuadrícula
ax.grid(True, which='both', linestyle='--', linewidth=0.7, color='grey', alpha = 0.5)

# Guardar el gráfico en una carpeta con buena resolución
plt.savefig(f"recons_1947_DT_KNN_RF.png", dpi=300, bbox_inches='tight')
plt.show()

#############################################################################

plt.figure(figsize=(15,7))
l1 = plt.plot(dfwork["temp_agua_CNN"].iloc[:365], color='cornflowerblue', label="Tª agua CNN")
plt.setp(l1, markersize=8)

l3 = plt.plot(dfwork["temp_agua_LR"].iloc[:365], color='seagreen', label="Tª agua LR")
plt.setp(l3, markersize=8)

l3 = plt.plot(dfwork["temp_agua_DT"].iloc[:365], color='red', label="Tª agua DT")
plt.setp(l3, markersize=8)

l4 = plt.plot(dfwork["temp_agua_KN"].iloc[:365], color='gold', label="Tª agua KNN")
plt.setp(l4, markersize=8)

l5 = plt.plot(dfwork["temp_agua_RF"].iloc[:365], color='indigo', label="Tª agua RF")
plt.setp(l5, markersize=8)


q = plt.plot(dfwork["ambiente"].iloc[:365] , color='darkgray', label="Tª ambiente")

# Configuración de los ejes y títulos
plt.xlabel('Fecha', fontsize=14)
plt.ylabel('Temperatura (ºC)', fontsize=14)
plt.title(f'Reconstrucción histórica 1947', fontsize=16)

# Quitar los bordes superior y derecho
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Ajustar el tamaño de la leyenda
plt.legend(fontsize=12)

# Ajustar el tamaño de las etiquetas de los ejes
ax.tick_params(axis='both', which='major', labelsize=12)

# Configurar la cuadrícula
ax.grid(True, which='both', linestyle='--', linewidth=0.7, color='grey', alpha = 0.5)

# Guardar el gráfico en una carpeta con buena resolución
plt.savefig(f"recons_1947_1.png", dpi=300, bbox_inches='tight')
plt.show()

#############################################################################

plt.figure(figsize=(15,7))
l1 = plt.plot(dfwork["temp_agua_CNN"].iloc[:365], color='cornflowerblue', label="Tª agua CNN")
plt.setp(l1, markersize=8)

l3 = plt.plot(dfwork["temp_agua_LR"].iloc[:365], color='seagreen', label="Tª agua LR")
# plt.setp(l3, markersize=8)

# l3 = plt.plot(dfwork["temp_agua_DT"].iloc[:365], color='red', label="Tª agua DT")
# plt.setp(l3, markersize=8)

l4 = plt.plot(dfwork["temp_agua_KN"].iloc[:365], color='gold', label="Tª agua KNN")
plt.setp(l4, markersize=8)

# l5 = plt.plot(dfwork["temp_agua_RF"].iloc[:365], color='indigo', label="Tª agua RF")
# plt.setp(l5, markersize=8)


q = plt.plot(dfwork["ambiente"].iloc[:365] , color='darkgray', label="Tª ambiente")

# Configuración de los ejes y títulos
plt.xlabel('Fecha', fontsize=14)
plt.ylabel('Temperatura (ºC)', fontsize=14)
plt.title(f'Reconstrucción histórica 1947', fontsize=16)

# Quitar los bordes superior y derecho
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Ajustar el tamaño de la leyenda
plt.legend(fontsize=12)

# Ajustar el tamaño de las etiquetas de los ejes
ax.tick_params(axis='both', which='major', labelsize=12)

# Configurar la cuadrícula
ax.grid(True, which='both', linestyle='--', linewidth=0.7, color='grey', alpha = 0.5)

# Guardar el gráfico en una carpeta con buena resolución
plt.savefig(f"recons_1947_2.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
colores = ["cornflowerblue", "seagreen","indigo","red","chocolate","orchid","darkgray","gold"]

plt.figure(figsize=(15,7))

q = plt.plot(dfwork["ambiente"] , color='darkgray', label="Temperatura ambiente", alpha = 1)
l1 = plt.plot(dfwork["temp_agua_CNN"], color='cornflowerblue', label="Temperatura agua CNN", alpha = 0.2)
plt.setp(l1, markersize=8)


# l3 = plt.plot(dfwork["temp_agua_LR"], color='seagreen', label="Temperatura agua LR", alpha = 0.5)
# plt.setp(l3, markersize=8)



# Configuración de los ejes y títulos
plt.xlabel('Fecha', fontsize=14)
plt.ylabel('Temperatura (ºC)', fontsize=14)
plt.title(f'Reconstrucción histórica ', fontsize=16)

# Quitar los bordes superior y derecho
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Ajustar el tamaño de la leyenda
plt.legend(fontsize=12)

# Ajustar el tamaño de las etiquetas de los ejes
ax.tick_params(axis='both', which='major', labelsize=12)

# Configurar la cuadrícula
ax.grid(True, which='both', linestyle='--', linewidth=0.7, color='grey', alpha = 0.5)

# Guardar el gráfico en una carpeta con buena resolución
plt.savefig(f"recons_hist.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
colores = ["cornflowerblue", "seagreen","indigo","red","chocolate","orchid","darkgray","gold"]

plt.figure(figsize=(15,7))


dfwork.iloc[21180:21545]

plt.title('Y_predict')
l1 = plt.plot(dfwork["temp_agua_CNN"].iloc[21180:21545], color='cornflowerblue', label="Tª agua CNN")
plt.setp(l1, markersize=8)

l3 = plt.plot(dfwork["temp_agua_LR"].iloc[21180:21545], color='seagreen', label="Tª agua LR")
plt.setp(l3, markersize=8)

# l3 = plt.plot(dfwork["temp_agua_DT"].iloc[21180:21545], color='red', label="Tª agua DT")
# plt.setp(l3, markersize=8)

l4 = plt.plot(dfwork["temp_agua_KN"].iloc[21180:21545], color='gold', label="Tª agua KNN")
plt.setp(l4, markersize=8)

# l5 = plt.plot(dfwork["temp_agua_RF"].iloc[21180:21545], color='indigo', label="Tª agua RF")
# plt.setp(l5, markersize=8)


q = plt.plot(dfwork["ambiente"].iloc[21180:21545] , color='darkgray', label="Tª ambiente")

# Configuración de los ejes y títulos
plt.xlabel('Fecha', fontsize=14)
plt.ylabel('Temperatura (ºC)', fontsize=14)
plt.title(f'Reconstrucción histórica 2005', fontsize=16)

# Quitar los bordes superior y derecho
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Ajustar el tamaño de la leyenda
plt.legend(fontsize=12)

# Ajustar el tamaño de las etiquetas de los ejes
ax.tick_params(axis='both', which='major', labelsize=12)

# Configurar la cuadrícula
ax.grid(True, which='both', linestyle='--', linewidth=0.7, color='grey', alpha = 0.5)

# Guardar el gráfico en una carpeta con buena resolución
plt.savefig(f"recons_2005.png", dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
import matplotlib.dates as mdates
dfwork.index = pd.to_datetime(dfwork.index)
# Extraer el mes y el día de las fechas
dfwork['mes_dia'] = dfwork.index.strftime('%m-%d')

dfwork['mes_dia'] = pd.to_datetime(dfwork.index.strftime('2000-%m-%d'))

df_comparacion  = dfwork[['mes_dia', 'temp_agua_CNN', 'temp_agua_RNN', 'temp_agua_LR', 'temp_agua_DT', 'temp_agua_KN', 'temp_agua_RF', 'ambiente']]

df_comparacion .set_index('mes_dia', inplace=True)

plt.figure(figsize=(15,7))


# dfwork.iloc[21180:21545]

plt.title('Y_predict')
l1 = plt.plot(df_comparacion["temp_agua_CNN"].iloc[:360], color='cornflowerblue', label="Tª agua CNN 1947")
plt.setp(l1, markersize=8)

l1 = plt.plot(df_comparacion ["temp_agua_CNN"].iloc[21180:21545], color='darkblue', label="Tª agua CNN 2005")
plt.setp(l1, markersize=8)

q = plt.plot(df_comparacion ["ambiente"].iloc[:360] , color='dimgray', label="Tª ambiente 1947")
q = plt.plot(df_comparacion ["ambiente"].iloc[21180:21545] , color='darkgray', label="Tª ambiente 2005")

# Configuración de los ejes y títulos
plt.xlabel('Fecha', fontsize=14)
plt.ylabel('Temperatura (ºC)', fontsize=14)
plt.title(f'Reconstrucción histórica 1947 vs 2005', fontsize=16)

# Quitar los bordes superior y derecho
ax = plt.gca()
ax.spines['top'].set_visible(False)
ax.spines['right'].set_visible(False)

# Ajustar el tamaño de la leyenda
plt.legend(fontsize=12)

# Ajustar el tamaño de las etiquetas de los ejes
ax.tick_params(axis='both', which='major', labelsize=12)

# Configurar la cuadrícula
ax.grid(True, which='both', linestyle='--', linewidth=0.7, color='grey', alpha = 0.5)

plt.gca().xaxis.set_major_locator(mdates.MonthLocator())
plt.gca().xaxis.set_major_formatter(mdates.DateFormatter('%b'))

# Guardar el gráfico en una carpeta con buena resolución
plt.savefig(f"recons_1947_2005.png", dpi=300, bbox_inches='tight')
plt.show()